SADC Tourism  management

In [1]:
from __future__ import annotations
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from datetime import date

In [5]:
class AvailabilityError(Exception):
  pass

class ValidationError(Exception):
  pass

@dataclass
class Country:
  name: str
  currency: str
  experiences: List['TourismExperience'] = field(default_factory= list)

  def add_experience(self, exp: 'TourismExperience') -> None:
    self.experiences.append(exp)


@dataclass
class TourismGroup:
  group_id: str
  members: int
  budget_per_person: float
  preferences: List[str]  # e.g., ['safari', 'beach']
  def total_budget(self) -> float:
    return self.members * self.budget_per_person

@dataclass
class TourismExperience:
  code:str
  title:str
  base_price: float
  capacity: int

  # Seasonal_price: mapping season -> multiplier ( e.g., high: 1.2)
  seasonal_price: Dict[str,float]=field(default_factory= lambda:{'low': 0.9, 'high': 1.2})
  booked_spots: int = 0

  def get_price(self, season: str = 'mid') -> float:
    multiplier = self.seasonal_price.get(season, 1)
    return round(self.base_price * multiplier,2)

  def available_spots(self) -> int:
    return max(0,self.capacity - self.booked_spots)

  def book(self, num_people: int) -> None:
    if self.available_spots() < num_people:
      raise AvailabilityError(f"Not enough availability on {self.title}.")
    self.booked_spots += num_people

@dataclass
class safari(TourismExperience):
  animals: List[str] = field(default_factory= list)

@dataclass
class BeachHoliday(TourismExperience):
  location: str = ""

@dataclass
class CulturalTour(TourismExperience):
  theme: str =""

@dataclass
class Package:
  """
  A tourism package which may include experience from single or multiple countries"""

  package_id: str
  experiences: List[TourismExperience]
  countries: List[Country]
  experiences: List[TourismExperience]
  bookings: Dict[str, int] = field(default_factory=dict)


  def total_price_per_person(self, season: str = 'mid') -> float:
    return round(sum(exp.get_price(season) for exp in self.experiences),2)

  def validate_cross_border(self) -> None:
    if not self.countries:
      raise ValidationError("Package must include at least one country.")
    names = [c.name for c in self.countries]
    if len(set(names)) != len(set(names)):
      raise ValidationError("Duplicate countries found in package.")

  def check_availability(self, num_people: int) -> bool:
    return all(exp.available_spots() >= num_people for exp in self.experiences)

  def book_for_group(self, group: TourismGroup, season: str = 'mid') -> float:

    """
    Book the package for a group of people and return total cost.
    Raises AvailabilityError if not enough availability.
    """

    self.validate_cross_border()
    if not self.check_availability(group.members):
      raise AvailabilityError("Not enough availability for all experiences.")

    # Simple budget check
    price_per_person = self.total_price_per_person(season)
    if price_per_person > group.budget_per_person:
      raise AvailabilityError(f"Price per person ({price_per_person}) exceeds group's budget ({group.budget_per_person}).")

    #Perfom Bookings
    for exp in self.experiences:
      exp.book(group.members)
    self.bookings[group.group_id] = group.members

    return price_per_person * group.members


Sample Usage

In [8]:
if __name__ == "__main__":
  #create Countries
  na = Country("Namibia", "NAD")
  sa = Country("South Africa", "ZAR")

  #Create experience
  kruger = safari("SA-SAF-001", "Kruger Classic Safari", base_price=250.0, capacity=20, animals=["lion", "elephant"])
  cape_beach = BeachHoliday("SA-BCH-001", "Cape Town Beach Relax", base_price=80.0, capacity=50, location="Clifton")
  etosha = safari("NA-SAF-001", "Etosha Safari", base_price=220.0, capacity=15, animals=["rhino", "giraffe"])

  #Add experience to countries
  na.add_experience(etosha)
  sa.add_experience(cape_beach)
  sa.add_experience(kruger)

  #seasonal tweak example
  kruger.seasonal_price = {'low': 0.8, 'high': 1.1}
  etosha.seasonal_price = {'low': 0.9, 'high': 1.2}

  #create a cross-border package (SA + Namibia)
  package = Package("PKG-001", experiences= [kruger, etosha], countries= [sa, na])

  #create a tourist group
  group = TourismGroup(group_id="GRP-001", members = 10, budget_per_person= 1000, preferences= ["safari", "beach"])

  #try booking

  try:
    total_cost = package.book_for_group(group, season = 'high')
    print(f"Booking successful! Total cost:R{total_cost}")

  except (AvailabilityError, ValidationError) as e:
    print("Booking failed", e)

Booking successful! Total cost:R5390.0
